##GAN&CLIP&GA - CIFAR 10 EVALUATION

Authors: Maciej Domagała, Adam Komorowski

### GENERATING IMAGES

In [1]:
#@title Initialization (Installation, Imports) - check github branch!
%%capture
import subprocess
import re

nvcc = subprocess.check_output(["nvcc", "--version"]).decode("utf-8")
version = re.findall("release (\d+\.\d+)", nvcc)[0]

pytorch_suffix = {
    "10.0": "+cu100",
    "10.1": "+cu101",
    "10.2": "",
}

pytorch_version = "1.7.1" + (pytorch_suffix[version] if version in pytorch_suffix else "+cu110")
torchvision_version = "0.8.2" + (pytorch_suffix[version] if version in pytorch_suffix else "+cu110")

!git clone https://github.com/maciejdomagala/clip_gans.git
%cd clip_gans
!git checkout main ### WARNING!!!

try:
  import torch
except:
  !pip install torch=={pytorch_version} -f https://download.pytorch.org/whl/torch_stable.html

try:
  import torchvision
except:
  !pip install torchvision=={torchvision_version} -f https://download.pytorch.org/whl/torch_stable.html

!pip install pytorch_pretrained_biggan pymoo kornia ftfy tensorboard

# IMPORTS
import warnings

import argparse
import os
import torch
import numpy as np
import pickle
import math
from tqdm import tqdm_notebook
from pymoo.optimize import minimize
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_algorithm, get_decision_making, get_decomposition
from pymoo.visualization.scatter import Scatter
import torchvision
from IPython.display import Image, display
import urllib.request
import cv2

from config import get_config
from problem import GenerationProblem
from operators import get_operators, get_algo


warnings.filterwarnings('ignore')

In [11]:
#@title Input Parameters (Code will produce (population_size*number_of_loops) evaluation images)
target = "deer" #@param {type:"string"}
config = "DeepMindBigGAN256_ga" #@param ["DeepMindBigGAN256_ga","DeepMindBigGAN256_de", "StyleGAN2_ffhq_ga","StyleGAN2_ffhq_de", "StyleGAN2_car_ga", "StyleGAN2_car_de","StyleGAN2_church_ga", "StyleGAN2_church_de","StyleGAN2_cat_ga", "StyleGAN2_cat_de"]
generations =   40#@param {type:"number"}
population_size = 64 #@param {type:"number"}
number_of_loops =  3#@param {type:"number"}

In [12]:
#@title Download weights for given model
!chmod +x ./download-weights.sh
if "ffhq" in config:
  ! ./download-weights.sh StyleGAN2-ffhq
if "church" in config:
  ! ./download-weights.sh StyleGAN2-church
if "car" in config:
  ! ./download-weights.sh StyleGAN2-car
if "cat" in config:
  ! ./download-weights.sh StyleGAN2-cat

chmod: cannot access './download-weights.sh': No such file or directory


In [13]:
#@title Config 
config = argparse.Namespace(
    config=config,
    target=target,
    device="cuda",
    generations=generations,
    tmp_folder="./results"
)
vars(config).update(get_config(config.config))
config.pop_size = population_size
# config.algorithm = algorithm_param
config.generations = generations

if not os.path.exists(config.tmp_folder): os.mkdir(config.tmp_folder)
if not os.path.exists(f'{config.tmp_folder}/{config.target}'): os.mkdir(f'{config.tmp_folder}/{config.target}')


In [14]:
#@title Support methods

def split_into_tiles_and_save(image):
  """
  Method splits batch image from the Pymoo output to the equal tiles and save them
  to the results directory.
  """
  global config, iteration, loop, target
  M = int(image.shape[0]//math.sqrt(config.pop_size))
  N = int(image.shape[1]//math.sqrt(config.pop_size))
  tiles = [image[x:x+M,y:y+N] for x in range(0,image.shape[0],M) for y in range(0,image.shape[1],N)]
  tiles = [tile for tile in tiles if tile.shape == (M,N,3)]
  if not os.path.exists(f'{config.tmp_folder}/{config.target}/{loop}'): os.mkdir(f'{config.tmp_folder}/{config.target}/{loop}')
  for j, each in enumerate(tiles):
    cv2.imwrite(f'./{config.tmp_folder}/{config.target}/{loop}/{target}_loop_{loop}_id_{j}.jpg', each)

def callback(algorithm):
    global config, iteration, loop, target

    iteration += 1

    sortedpop = sorted(algorithm.pop, key=lambda p: p.F)
    X = np.stack([p.X for p in sortedpop])      
    ls = config.latent(config)
    ls.set_from_population(X)
    if iteration == config.generations:
      with torch.no_grad():
        generated = algorithm.problem.generator.generate(ls)
        name = f'batch_loop_{loop}.jpg'
        algorithm.problem.generator.save(generated, f"./{config.tmp_folder}/{config.target}/{name}")
        im = cv2.imread(f"./{config.tmp_folder}/{config.target}/{name}")
        split_into_tiles_and_save(im)

def generate_image(loop: int):
  problem = GenerationProblem(config)

  kwargs = get_algo(config)

  algorithm = get_algorithm(
      config.algorithm,
      callback=callback,
      **kwargs
  )

  res = minimize(
      problem,
      algorithm,
      ("n_gen", config.generations),
      save_history=False,
      verbose=True,
  )

In [15]:
#@title RUN (GENERATE IMAGES)


for loop in tqdm_notebook(range(number_of_loops)):
  iteration=0
  generate_image(loop)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |     fopt     |     favg    
    1 |      64 |  0.00000E+00 |  0.00000E+00 |      -0.2505 |      -0.2174
    2 |     128 |  0.00000E+00 |  0.00000E+00 |      -0.2595 |      -0.2314
    3 |     192 |  0.00000E+00 |  0.00000E+00 |      -0.2703 |       -0.238
    4 |     256 |  0.00000E+00 |  0.00000E+00 |      -0.2703 |      -0.2411
    5 |     320 |  0.00000E+00 |  0.00000E+00 |      -0.2715 |      -0.2451
    6 |     384 |  0.00000E+00 |  0.00000E+00 |      -0.2964 |      -0.2505
    7 |     448 |  0.00000E+00 |  0.00000E+00 |      -0.2964 |      -0.2544
    8 |     512 |  0.00000E+00 |  0.00000E+00 |      -0.2964 |      -0.2563
    9 |     576 |  0.00000E+00 |  0.00000E+00 |      -0.2964 |      -0.2593
   10 |     640 |  0.00000E+00 |  0.00000E+00 |      -0.2996 |      -0.2625
   11 |     704 |  0.00000E+00 |  0.00000E+00 |      -0.2996 |      -0.2651
   12 |     768 |  0.00000E+00 |  0.00000E+00 |      -0.2996 |      -0.2676
   13 |     

### CIFAR10 - CLASS EVALUATION

In [9]:
#@title Initialization
%%capture

%cd /content
!git clone https://github.com/vrakesh/CIFAR-10-Classifier
%cd CIFAR-10-Classifier/

from cifar_classifier import cifar_classifier
cc = cifar_classifier()
model = cc.get_model('cnn')
model.load_weights('cifar10.model.cnn.hdf5')
cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [10]:
#@title RUN - EVALUATE CLASS
class_to_check = 'deer' #@param {type:"string"}
import PIL
folders_to_check = [dI for dI in os.listdir(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}') if os.path.isdir(os.path.join(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}', dI))]
list_of_images = list()
for folder in folders_to_check:
  for (dirpath, dirnames, filenames) in os.walk(f'/content/clip_gans/{config.tmp_folder[2:]}/{class_to_check}/{folder}'):
      list_of_images += [os.path.join(dirpath, file) for file in filenames]

acc_count = 0
for image_path in tqdm_notebook(list_of_images):
  im = (PIL.Image.open(image_path))
  im = im.resize((32, 32))
  im = np.expand_dims(im, 0)
  y_hat = model.predict(im)
  predicted_label = cifar10_labels[np.argmax(y_hat[0])]
  if predicted_label==class_to_check:
    acc_count += 1
print(f'\nAccuracy for {class_to_check.upper()} class: {round(acc_count/len(list_of_images)*100, 2)}%')
print(f'Number of evaluated images: {len(list_of_images)}')
print("")
print(f'Generation process summary:')
print(f'- Number of iterations per loop: {config.generations}')

FileNotFoundError: ignored

###CIFAR10 - GENERATE & EVALUATE ALL CLASSES

**Note**. To run cells below you need to execute the following cells:
* Initialization
* Support methods
* Cifar10 - Initialization

In [17]:
#@title RUN ALL CIFAR10 CLASSES EVALUATION 

!rm -r /content/clip_gans/results/*

import PIL

# target = "deer" #@param {type:"string"}
config = "DeepMindBigGAN256_ga" #@param ["DeepMindBigGAN256_ga","DeepMindBigGAN256_de", "StyleGAN2_ffhq_ga","StyleGAN2_ffhq_de", "StyleGAN2_car_ga", "StyleGAN2_car_de","StyleGAN2_church_ga", "StyleGAN2_church_de","StyleGAN2_cat_ga", "StyleGAN2_cat_de"]
generations =  10#@param {type:"number"}
population_size = 64 #@param {type:"number"}
number_of_loops =  1#@param {type:"number"}
algorithm_param = 'ga' #@param ['ga', 'de']

%cd /content/clip_gans

config = argparse.Namespace(
    config=config,
    device="cuda",
    generations=generations,
    tmp_folder="./results"
)
vars(config).update(get_config(config.config))
config.pop_size = population_size
config.algorithm = algorithm_param
config.generations = generations

if not os.path.exists(config.tmp_folder): os.mkdir(config.tmp_folder)


for target in cifar10_labels:
  print(f'\n{target} started.')
  config.target = target
  if not os.path.exists(f'{config.tmp_folder}/{config.target}'): os.mkdir(f'{config.tmp_folder}/{config.target}')
  for loop in range(number_of_loops):
    iteration=0
    generate_image(loop)
  print(f'{target} finished.')

print("")

for class_to_check in cifar10_labels:
  folders_to_check = [dI for dI in os.listdir(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}') if os.path.isdir(os.path.join(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}', dI))]
  list_of_images = list()
  for folder in folders_to_check:
    for (dirpath, dirnames, filenames) in os.walk(f'/content/clip_gans/{config.tmp_folder[2:]}/{class_to_check}/{folder}'):
        list_of_images += [os.path.join(dirpath, file) for file in filenames]

  acc_count = 0
  for image_path in list_of_images:
    im = (PIL.Image.open(image_path))
    im = im.resize((32, 32))
    im = np.expand_dims(im, 0)
    y_hat = model.predict(im)
    predicted_label = cifar10_labels[np.argmax(y_hat[0])]
    if predicted_label==class_to_check:
      acc_count += 1
  print(f'{class_to_check.upper()} accuracy: {round(acc_count/len(list_of_images)*100, 2)}%, ({acc_count}/{len(list_of_images)})')

rm: cannot remove '/content/clip_gans/results/*': No such file or directory
/content/clip_gans

airplane started.
n_gen |  n_eval |   cv (min)   |   cv (avg)   |     fopt     |     favg    
    1 |      64 |  0.00000E+00 |  0.00000E+00 |      -0.2354 |      -0.2084
    2 |     128 |  0.00000E+00 |  0.00000E+00 |      -0.2365 |      -0.2211
    3 |     192 |  0.00000E+00 |  0.00000E+00 |       -0.245 |      -0.2253
    4 |     256 |  0.00000E+00 |  0.00000E+00 |      -0.2483 |      -0.2289
    5 |     320 |  0.00000E+00 |  0.00000E+00 |      -0.2483 |      -0.2318
    6 |     384 |  0.00000E+00 |  0.00000E+00 |       -0.258 |      -0.2354
    7 |     448 |  0.00000E+00 |  0.00000E+00 |      -0.2607 |       -0.238
    8 |     512 |  0.00000E+00 |  0.00000E+00 |      -0.2607 |      -0.2397
    9 |     576 |  0.00000E+00 |  0.00000E+00 |      -0.2607 |       -0.242
   10 |     640 |  0.00000E+00 |  0.00000E+00 |       -0.262 |      -0.2435
airplane finished.

automobile started.
n_gen |  n